In [1]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
portfolio = pd.read_excel(io="Workshop Data.xlsx",sheet_name="Portfolio")
TenYearYield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="10yUST Yields")
HYyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="HY Index")
IGyield = pd.read_excel(io="Indexes and Spreads Data 01.09.xlsx",sheet_name="IG Index")
MoveIdx = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="MOVE Index")
Vix = pd.read_excel(io="MOVE Vix prices.xlsx",sheet_name="VIX")
SPY_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPY")
SPXT_Idx = pd.read_excel(io="SP500 Index data.xlsx",sheet_name="SPXT")

portfolio = portfolio.set_index("Date")
TenYearYield = TenYearYield.set_index("Date").sort_index()
HYyield = HYyield.set_index("Date").sort_index()
IGyield = IGyield.set_index("Date").sort_index()
MoveIdx = MoveIdx.set_index("Date").sort_index()
Vix = Vix.set_index("Date").sort_index()
SPY_Idx = SPY_Idx.set_index("Date").sort_index()
SPXT_Idx = SPXT_Idx.set_index("Date").sort_index()

portfolio.loc[:,"SPYCumDiv"]=portfolio.loc[:,"SPY Dividends"][::-1].cumsum()[::-1]
portfolio.loc[:,"TotalValueSPY"]=portfolio.loc[:,"SPY Position"]+portfolio.loc[:,"SPYCumDiv"]
portfolio = portfolio.sort_index()

SPYCorrData = pd.DataFrame(portfolio.loc["2021-01-11":"2025-12-31","TotalValueSPY"]).merge(HYyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
SPYCorrData.loc[:,"TotalReturnsSPY"] = SPYCorrData["TotalValueSPY"].pct_change()
SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(HYyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"HY_Index_OAS","INDEX_Z_SPREAD_BP":"HY_INDEX_Z_SPREAD"})
SPYCorrData.loc[:,"HY_Index_OAS_Diff"]=SPYCorrData["HY_Index_OAS"].diff()
SPYCorrData.loc[:,"HY_INDEX_Z_SPREAD_Diff"]=SPYCorrData["HY_INDEX_Z_SPREAD"].diff()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(TenYearYield.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"10y_UST_INDEX_PX"})
SPYCorrData.loc[:,"10yYieldDiff"] = SPYCorrData["10y_UST_INDEX_PX"].diff()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","OAS_SOVEREIGN_CURVE"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"OAS_SOVEREIGN_CURVE":"IG_Index_OAS"})
SPYCorrData.loc[:,"IG_Index_OAS_Diff"]=SPYCorrData["IG_Index_OAS"].diff()
SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(IGyield.loc["2021-01-11":"2025-12-31","INDEX_Z_SPREAD_BP"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"INDEX_Z_SPREAD_BP":"IG_INDEX_Z_SPREAD"})
SPYCorrData.loc[:,"IG_Index_Z_SPREAD_Diff"]=SPYCorrData["IG_INDEX_Z_SPREAD"].diff()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(SPY_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"SPY_Idx_LAST_PX"})
SPYCorrData.loc[:,"ReturnSPY_Idx"] = SPYCorrData["SPY_Idx_LAST_PX"].pct_change()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(SPXT_Idx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"SPXT_Idx_LAST_PX"})
SPYCorrData.loc[:,"ReturnSPXT_Idx"] = SPYCorrData["SPXT_Idx_LAST_PX"].pct_change()

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(MoveIdx.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"MOVE Idx"})

SPYCorrData = SPYCorrData.loc["2021-01-11":"2025-12-31"].merge(Vix.loc["2021-01-11":"2025-12-31","PX_LAST"],left_index=True,right_index=True)
SPYCorrData = SPYCorrData.rename(columns={"PX_LAST":"Vix Idx"})

SPYCorrData=SPYCorrData.dropna()
SPYCorrData = SPYCorrData.sort_index(ascending=False)

In [3]:
SPYCorrData[["TotalReturnsSPY","ReturnSPXT_Idx","ReturnSPY_Idx"]]

,TotalReturnsSPY,ReturnSPXT_Idx,ReturnSPY_Idx
Date,,,
2025-12-31,-0.007032,-0.007280,-0.007358
2025-12-30,-0.001123,-0.001374,-0.001376
2025-12-29,-0.003361,-0.003461,-0.003492
2025-12-26,-0.000053,-0.000209,-0.000304
2025-12-24,0.003404,0.003224,0.003222
...,...,...,...
2021-01-19,0.007917,0.008175,0.008136
2021-01-15,-0.007225,-0.007177,-0.007190
2021-01-14,-0.003435,-0.003638,-0.003753


In [4]:
SPYCorrData[["TotalReturnsSPY","ReturnSPXT_Idx","ReturnSPY_Idx"]].corr()

,TotalReturnsSPY,ReturnSPXT_Idx,ReturnSPY_Idx
TotalReturnsSPY,1.000000,0.997838,0.997819
ReturnSPXT_Idx,0.997838,1.000000,0.999984
ReturnSPY_Idx,0.997819,0.999984,1.000000


In [5]:
#Z-Score adjustment Process

X_Std = StandardScaler().fit_transform(SPYCorrData[["ReturnSPY_Idx","Vix Idx"]])
pcav1 = PCA()
pcav1.fit(X_Std)

print(pcav1.explained_variance_ratio_)
print(pd.DataFrame(pcav1.components_, columns=["ReturnSPY_Idx","Vix Idx"]))

X_pca = pcav1.transform(X_Std)

SPYCorrData.loc[:,"PC1"]=X_pca[:,0]
SPYCorrData.loc[:,"PC2"]=X_pca[:,1]


X = sm.add_constant(SPYCorrData[["PC1","PC2"]])
Y = SPYCorrData["TotalReturnsSPY"]

model = sm.OLS(Y,X).fit()
model.summary()

[0.60384755 0.39615245]
   ReturnSPY_Idx   Vix Idx
0      -0.707107  0.707107
1       0.707107  0.707107


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        TotalReturnsSPY   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                 1.415e+05
Date:                Fri, 16 Jan 2026   Prob (F-statistic):               0.00
Time:                        19:24:55   Log-Likelihood:                 7258.9
No. Observations:                1240   AIC:                        -1.451e+04
Df Residuals:                    1237   BIC:                        -1.450e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0005   1.97e-05     27.689      0.000       0.001       0.001
PC1           -0.0074    1.8e-05   -412.647      0.000      -0.007      -0.007
PC2            0.0074   2.22e-05    335.818      0.000       0.007       0.007
==============================================================================
Omnibus:                      967.592   Durbin-Watson:                   2.466
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            54984.261
Skew:                          -3.097   Prob(JB):                         0.00
Kurtosis:                      35.029   Cond. No.                         1.23
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""